In [2]:
import numpy as np
from Observer.EKF import EKF

KalmanFilter = EKF()

print(KalmanFilter._xhat)


[0.]


In [17]:
# Create fictional system 
A = np.array([[0,1,0],
            [-1,-2,-0.5],
            [0,0,0]])
B = np.array([0,0.5,0]).T
C = np.array([1,0,0])
D = 0
E = np.array([0,0,1]).T

